In [1]:
%%time
import sys
import polars as pl
import numpy as np
import pandas as pd

file, skiprows = "../vcf/sorghum.filtered.season4.season6.vcf.gz", 82
vcf = pl.read_csv(file, sep="\t", skip_rows=skiprows, dtypes={'#CHROM':pl.datatypes.Utf8})

contigs = vcf.select([pl.col("#CHROM").unique()])["#CHROM"].to_list()
contigs.sort()

for contig in contigs:
    vcf_w_list = vcf.filter(pl.col("#CHROM") == contig) \
        .select(pl.all().exclude(["#CHROM", "REF", "ALT", "POS", "QUAL", "FILTER", "INFO", "FORMAT"])) \
        .with_columns(pl.col("^PI.*$").str.replace("0\\|0","0.0")) \
        .with_columns(pl.col("^PI.*$").str.replace("1\\|0","0.5")) \
        .with_columns(pl.col("^PI.*$").str.replace("0\\|1","0.5")) \
        .with_columns(pl.col("^PI.*$").str.replace("1\\|1","1.0")) \
        .with_columns(pl.col("^PI.*$").cast(pl.Float32, strict=True)) \
        .select(["ID", pl.concat_list(pl.all().exclude("ID").alias('GENOTYPES'))])

    pandas_df = vcf_w_list.to_pandas()
    pandas_df['matrix'] = pandas_df.apply(lambda row: np.subtract(1,np.abs(np.subtract.outer(row['GENOTYPES'],row['GENOTYPES']))), axis=1)
    pandas_df.drop('GENOTYPES', axis=1, inplace=True)
    pandas_df.to_pickle(f'{contig}-matrix.pickle')


CPU times: user 2min 39s, sys: 3min 57s, total: 6min 36s
Wall time: 12min 54s


In [4]:
%%time
!pigz *-matrix.pickle

CPU times: user 10.5 s, sys: 2.95 s, total: 13.4 s
Wall time: 13min 40s
zsh:1: no matches found: *-matrix.pickle
CPU times: user 1.15 ms, sys: 5.09 ms, total: 6.24 ms
Wall time: 142 ms
